### Install packages

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("dada2")
BiocManager::install("ShortRead")
install.packages("magrittr")
install.packages("stringr")
install.packages("data.table")
install.packages("qualpalr")
install.packages("seqinr")

### Load libraries

In [ ]:
library(dada2)
library(stringr)
library(data.table)
library(qualpalr)
library(ShortRead)
library(Biostrings)
library(seqinr)

### File path setup

In [ ]:
rawpath <- "raw"
wdpath <- "/home/allie/domhain/2023-Longitudinal_oral_microbiome/01-read_processing" # change to where git repository was cloned
fnFs <- sort(list.files(rawpath, pattern="_R1_001.fastq.gz", full.names=T))
fnRs <- sort(list.files(rawpath, pattern="_R2_001.fastq.gz", full.names=T))
sample.names <- sapply(strsplit(basename(fnFs), "_"), `[`, 1)
head(sample.names, 10)
paste("Number of input samples: ", length(sample.names))

### Plot quality scores

In [ ]:
system("mkdir img") # ignore warning
fwdqual <- plotQualityProfile(fnFs[100:115])
revqual <- plotQualityProfile(fnRs[100:115])
fwdqual
revqual

pdf(paste(wdpath, "/img/", "forward_quality_plot.pdf", sep=""))
fwdqual
dev.off()
pdf(paste(wdpath, "/img/", "reverse_quality_plot.pdf", sep=""))
revqual
dev.off()

### Preliminary filter (remove sequences with uncalled bases)

In [ ]:
fnFs.filtN <- file.path(rawpath, "filtN", basename(fnFs)) # Put N-filtered files in filtN/ subdirectory
fnRs.filtN <- file.path(rawpath, "filtN", basename(fnRs))
filterAndTrim(fnFs, fnFs.filtN, fnRs, fnRs.filtN, maxN = 0, multithread = TRUE, compress = TRUE, matchIDs=TRUE)

### Primer trimming

In [ ]:
cutadapt <- as.character(system("which cutadapt", intern=T))
system("cutadapt --version")
path.cut <- file.path(rawpath, "cutadapt")
if(!dir.exists(path.cut)) dir.create(path.cut)
fnFs.cut <- file.path(path.cut, basename(fnFs))
fnRs.cut <- file.path(path.cut, basename(fnRs))
FWD.RC <- dada2:::rc("MAYGARAARMGNATGYTNCARGA")
REV.RC <- dada2:::rc("GMCATYTGRTCNCCRTCRAA")
# Trim FWD and the reverse-complement of REV off of R1 (forward reads)
R1.flags <- paste("-g", "MAYGARAARMGNATGYTNCARGA", "-a", REV.RC) 
# Trim REV and the reverse-complement of FWD off of R2 (reverse reads)
R2.flags <- paste("-G", "GMCATYTGRTCNCCRTCRAA", "-A", FWD.RC) 
for(i in seq_along(fnFs)) {
  system2(cutadapt, args = c("--cores=0", R1.flags, R2.flags, "-n", 2,"-o", fnFs.cut[i], "-p", fnRs.cut[i], fnFs.filtN[i], fnRs.filtN[i]))
}
cutFs <- sort(list.files(path.cut, pattern = "R1", full.names = TRUE))
cutRs <- sort(list.files(path.cut, pattern = "R2", full.names = TRUE))

### Quality filter and trim reads

In [ ]:
filtFs <- file.path(path.cut, "filtered", basename(cutFs))
filtRs <- file.path(path.cut, "filtered", basename(cutRs))
out <- filterAndTrim(cutFs, filtFs, cutRs, filtRs, trimRight=25, maxN=c(0,0), maxEE=c(4,6), rm.phix=TRUE, matchIDs=TRUE, compress=TRUE, multithread=TRUE)
retained <- as.data.frame(out)
# aiming for around 80% or more retention
retained$prop.postqual <- retained$reads.out/retained$reads.in*100
head(retained)
mean(retained$prop.postqual)
median(retained$prop.postqual)

### Learn and plot error rates

In [ ]:
set.seed(12349)
errF <- learnErrors(filtFs, multithread=T, random=T)
errR <- learnErrors(filtRs, multithread=T, random=T)
err.f.plt <- plotErrors(errF, nominalQ=TRUE) 
err.f.plt
pdf(paste(wdpath, "/img/", "error_plot.pdf", sep=""))
err.f.plt
dev.off()

### Dereplication

In [ ]:
derepFs <- derepFastq(filtFs, verbose=FALSE)
derepRs <- derepFastq(filtRs, verbose=FALSE)
# reassign sample names
sample.names <- sapply(strsplit(basename(filtFs), "_"), `[`, 1)
names(derepFs) <- sample.names
names(derepRs) <- sample.names

### Sample inference

In [ ]:
dadaFs <- dada(derepFs, err=errF, multithread=TRUE, verbose=FALSE)
dadaRs <- dada(derepRs, err=errR, multithread=TRUE, verbose=FALSE)

### Merge paired-end reads

In [ ]:
mergers <- mergePairs(dadaFs, derepFs, dadaRs, derepRs, verbose=FALSE)

### Construct sequence table

In [ ]:
seqtab <- makeSequenceTable(mergers)
dim(seqtab)

### Length filter

In [ ]:
# sequence length table
table(nchar(colnames(seqtab)))

Filter out any ASVs shorter than 450 bp in length

In [ ]:
seqlens <- nchar(getSequences(seqtab))
seqtab.filt <- seqtab[,seqlens > 450]
dim(seqtab.filt)

Sequence length distribution after filter

In [ ]:
length.histogram <- as.data.frame(table(nchar(getSequences(seqtab.filt))))
plot(x=length.histogram[,1], y=length.histogram[,2])
pdf(paste(wdpath, "/img/", "length_hist.pdf", sep=""))
plot(x=length.histogram[,1], y=length.histogram[,2])
dev.off()

### Chimera detection and removal

In [ ]:
seqtab.nochim <- removeBimeraDenovo(seqtab.filt, method="pooled", multithread=T, verbose=T)
dim(seqtab.nochim)
sum(seqtab.nochim)/sum(seqtab.filt)

### Processing summary

In [ ]:
getN <- function(x) sum(getUniques(x))
track <- cbind(out, sapply(dadaFs, getN), sapply(dadaRs, getN), sapply(mergers, getN), rowSums(seqtab.nochim))
colnames(track) <- c("input", "filtered", "denoisedF", "denoisedR", "merged", "nochimeras")
rownames(track) <- sample.names
track

### Save output

In [ ]:
write.table(data.frame("row_names"=rownames(track),track),"read_retention.txt", row.names=FALSE, quote=F, sep="\t")
uniquesToFasta(seqtab.nochim, "rep_set.fa")
system("awk '/^>/{print \">ASV\" ++i; next}{print}' < rep_set.fa > rep_set_fix.fa")
system("mv rep_set_fix.fa rep_set.fa")

### Clean up ASV names

In [ ]:
my_otu_table <- t(as.data.frame(seqtab.nochim)) 
ASV.seq <- as.character(unclass(row.names(my_otu_table))) 
ASV.num <- paste0("ASV", seq(ASV.seq), sep='') 
colnames(seqtab.nochim) <- ASV.num 
write.table(data.frame("row_names"=rownames(seqtab.nochim),seqtab.nochim),"sequence_table.merged.txt", row.names=FALSE, quote=F, sep="\t")